In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!pip install wikipedia-api

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia-api: filename=Wikipedia_API-0.8.1-py3-none-any.whl size=15384 sha256=a91bc47abb0ea28ed737175df5d79d6080c2ecda36c6dab7346e79910533cf69
  Stored in directory: /root/.cache/pip/wheels/0b/0f/39/e8214ec038ccd5aeb8c82b957289f2f3ab2251febeae5c2860
Successfully built wikipedia-api


In [3]:
!pip install -q -U google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.9/165.9 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 6.0 MB/s eta 0:00:00


# Making the Questions Constant For Each Category

In [ ]:
import pandas as pd
from google import genai
import wikipediaapi

# !pip install wikipedia-api
# !pip install -q -U google-genai

wiki_wiki = wikipediaapi.Wikipedia(user_agent='MyProjectName (merlin@example.com)', language='en') # Wikipedia Api setup
client = genai.Client(api_key="<API>") # Gemini Api setup

In [7]:
data = pd.read_csv("/kaggle/input/qa-dataset/mapped_qa_data.csv")
data.head()

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Index,Concept,Question,Answer,Photo,photo_path
0,73.0,Bab_al-Nasr_(Cairo),Is this in Cairo?,"Yes, Cairo.",NaN,/downloaded_images/img_0.jpg
1,73.0,Bab_al-Nasr_(Cairo),What structure is this?,Gate of Victory,NaN,/downloaded_images/img_1.jpg
2,73.0,Bab_al-Nasr_(Cairo),Is this a gate?,"Yes, gate.",NaN,/downloaded_images/img_2.jpg
3,73.0,Bab_al-Nasr_(Cairo),What year was it built?,1087 AD,NaN,/downloaded_images/img_3.jpg
4,73.0,Bab_al-Nasr_(Cairo),Who built this gate?,Badr al-Jamali,NaN,/downloaded_images/img_4.jpg


In [9]:
data_concepts = data.iloc[:,:2].drop_duplicates().set_index('Index')
data_concepts

,Concept
Index,
73.0,Bab_al-Nasr_(Cairo)
74.0,Mortuary_Temple_of_Seti_I_in_Qurna
75.0,Great_Pyramid_of_Giza
76.0,Umm_Kulthum_Museum
77.0,Wadi_el-Raiyan
...,...
347.0,Sohag_Stadium
348.0,Userkaf_sun_temple
349.0,Ras_el-Tin_Palace


In [11]:
import time

concept_category_map = []
for concept in data_concepts.loc[:,'Concept']:
    try:
        concept_wiki = wiki_wiki.page(concept)
        category = "NaN"
        if not concept_wiki.exists():
            words = concept.split('_')
            concept_wiki = wiki_wiki.page('_'.join(words[:-1]))
            # Loop and remove words from end
            while not concept_wiki.exists and len(words):
                words.pop()
                concept_wiki = wiki_wiki.page('_'.join(words[:-1]))
                time.sleep(1)
                
        if len(concept.split('_')):
            concept_wiki_summary = "Wikipedia Summary: '" + concept_wiki.summary + "'"
            #print(concept_wiki_summary)
            prompt = \
            f"""
            Categorize {concept} Using your prior information and this provided wikipedia page summary:
            {concept_wiki_summary}
            into one of the following categories.
            ## Categories
            - Festivals/Music/Dance  (Unique festival celebration scenes, Traditional musical instruments, dance scenes)
            - Handicraft Artifacts (Ethnic handicrafts, traditional handmade items)
            - Traditional Clothing (Ethnic clothing, festival attire)
            - Religion and belief (Modern Temples and churches, religious ceremonies)
            - Famous Landmarks (Famous historical sites, Ancient Temples, buildings)
            - Historical Artifacts (Museum collections, ancient relics)
            - Historical Figures (Portraits of historical figures, statues)
            - Food (Local specialty dishes, traditional festival foods)
            - Natural scenery and Ecosystem (Unique natural landscapes, ecological reserves)
    
            The Output response should only be the category.
            """
            #print("/n"+prompt)
            category = client.models.generate_content(model="gemini-2.0-flash", contents=prompt).text.strip('\n')
            time.sleep(5)
    except Exception as e:
        concept_wiki_summary = f"Error fetching page '{concept}': {str(e)}"
            
    concept_category_map.append({"Concept":concept, "Category": category})
            
concept_category_map = pd.DataFrame(concept_category_map)
concept_category_map

,Concept,Category
0,Bab_al-Nasr_(Cairo),Famous Landmarks
1,Mortuary_Temple_of_Seti_I_in_Qurna,Famous Landmarks
2,Great_Pyramid_of_Giza,Famous Landmarks
3,Umm_Kulthum_Museum,Historical Figures
4,Wadi_el-Raiyan,Natural scenery and Ecosystem
...,...,...
275,Sohag_Stadium,Famous Landmarks
276,Userkaf_sun_temple,Famous Landmarks
277,Ras_el-Tin_Palace,Famous Landmarks
278,"Tirbana_mosque,_Alexandria",Religion and belief


In [12]:
concept_category_map.to_csv("concept_category_map.csv")

In [13]:
data

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Index,Concept,Question,Answer,Photo,photo_path
0,73.0,Bab_al-Nasr_(Cairo),Is this in Cairo?,"Yes, Cairo.",NaN,/downloaded_images/img_0.jpg
1,73.0,Bab_al-Nasr_(Cairo),What structure is this?,Gate of Victory,NaN,/downloaded_images/img_1.jpg
2,73.0,Bab_al-Nasr_(Cairo),Is this a gate?,"Yes, gate.",NaN,/downloaded_images/img_2.jpg
3,73.0,Bab_al-Nasr_(Cairo),What year was it built?,1087 AD,NaN,/downloaded_images/img_3.jpg
4,73.0,Bab_al-Nasr_(Cairo),Who built this gate?,Badr al-Jamali,NaN,/downloaded_images/img_4.jpg
...,...,...,...,...,...,...
6249,351.0,Abu_Ghurab,What is celebrated?,Sun god Ra.,NaN,/downloaded_images/img_5388.JPG
6250,351.0,Abu_Ghurab,Are offering tables present?,"Yes, offering tables present.",NaN,/downloaded_images/img_5389.jpg
6251,351.0,Abu_Ghurab,What is the temple's function?,Solar cult.,NaN,/downloaded_images/img_5390.jpg
6252,351.0,Abu_Ghurab,What does it provide insight into?,Religious beliefs.,NaN,/downloaded_images/img_5391.JPG


In [26]:
mapped_qa_category = data.merge(concept_category_map, on='Concept')
mapped_qa_category_data = mapped_qa_category[["Index", "Concept", "Category", "Question", "Answer"]]
mapped_qa_category_data["Index"] = mapped_qa_category_data["Index"].fillna(198.0).astype(int)
mapped_qa_category_data

/tmp/ipykernel_31/902915522.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mapped_qa_category_data["Index"] = mapped_qa_category_data["Index"].fillna(198.0).astype(int)


,Index,Concept,Category,Question,Answer
0,73,Bab_al-Nasr_(Cairo),Famous Landmarks,Is this in Cairo?,"Yes, Cairo."
1,73,Bab_al-Nasr_(Cairo),Famous Landmarks,What structure is this?,Gate of Victory
2,73,Bab_al-Nasr_(Cairo),Famous Landmarks,Is this a gate?,"Yes, gate."
3,73,Bab_al-Nasr_(Cairo),Famous Landmarks,What year was it built?,1087 AD
4,73,Bab_al-Nasr_(Cairo),Famous Landmarks,Who built this gate?,Badr al-Jamali
...,...,...,...,...,...
6266,351,Abu_Ghurab,Famous Landmarks,What is celebrated?,Sun god Ra.
6267,351,Abu_Ghurab,Famous Landmarks,Are offering tables present?,"Yes, offering tables present."
6268,351,Abu_Ghurab,Famous Landmarks,What is the temple's function?,Solar cult.
6269,351,Abu_Ghurab,Famous Landmarks,What does it provide insight into?,Religious beliefs.


In [28]:
mapped_qa_category_data.to_csv("mapped_qa_category_data.csv")

In [30]:
category_question_map = {
    "Festivals/Music/Dance" : ["Which festival is celebrated",
                               "Which city hosts it",
                                "When is it held",
                                "What dance style",
                                "What instrument played",
                                "What is the costume",
                                "What cultural group celebrates it",
                                "Which deity honored",
                                "What ritual happens",
                                "What season it symbolizes",],
    "Handicraft Artifacts" : ["What is the artifact name",
                                "Which material is used",
                                "What technique is applied",
                                "Which region is the origin",
                                "What cultural group",
                                "What is the decorative motif",
                                "Which purpose it serves",
                                "What color palette",
                                "What time period",
                                "What symbolic meaning",],
    "Traditional Clothing" : ["What is the clothes name",
                                "What garment type",
                                "Which region origin",
                                "What main fabric",
                                "Which pattern featured",
                                "What primary color",
                                "Which occasion worn",
                                "What accessory worn",
                                "Which era style",
                                "What cultural significance",
                                "What gender wears it",], 
    "Religion and belief" : ["Which religion represented",
                                "Which deity worshipped",
                                "What ritual performed",
                                "Which ceremony type",
                                "What offering present",
                                "Which religious symbol",
                                "What scripture shown",
                                "Which holy place",
                                "What architectural style",
                                "Which festival shown",],
    "Famous Landmarks" : ["Where exactly located",
                                "When established",
                                "Who built it",
                                "Which ruler commissioned",
                                "Which architectural style",
                                "What original name",
                                "What primary material",
                                "What purpose served",],
    "Historical Artifacts" : ["What object type",
                                "Which era dates",
                                "Which civilization made",
                                "What material crafted",
                                "Which purpose served",
                                "What inscription reads",
                                "What decorative motif",
                                "Which museum houses",
                                "When discovered",
                                "What condition status",],
    "Historical Figures" : ["Who is depicted",
                                "Which role held",
                                "Which era lived",
                                "Which nationality",
                                "What attire worn",
                                "What expression shown",
                                "What title inscribed",
                                "Which headgear worn",
                                "What symbolic object",
                                "Which achievement noted",],
    "Food" : ["Which dish shown",
                "What main ingredient",
                "Which cooking method",
                "What region specialty",
                "Which festival served",
                "What garnish used",
                "What flavor profile",
                "What serving vessel",
                "What color topping",
                "Which side dish",],
    "Natural scenery and Ecosystem" : ["Which landscape type",
                                            "Which biome present",
                                            "Which region located",
                                            "What season shown",
                                            "Which water body",
                                            "Which mountain range",
                                            "What vegetation type",
                                            "Which wildlife species",
                                            "What protection status",
                                            "Which ecological reserve",],  
}

In [47]:
const_qa = []
for row in concept_category_map.itertuples(index=False):
    concept = row.Concept
    category = row.Category
    const_qa.append({
        "Concept":concept,
        "Question": "What Cultural Category does the object in the image belong to",
        "Answer": category
    })
    try:
        for q in category_question_map[category]:
            const_qa.append({
                "Concept":concept,
                "Question": q,
                "Answer": None
            })
    except:
        pass

const_qa_data = pd.DataFrame(const_qa)
const_qa_data

,Concept,Question,Answer
0,Bab_al-Nasr_(Cairo),What Cultural Category does the object in the ...,Famous Landmarks
1,Bab_al-Nasr_(Cairo),Where exactly located,None
2,Bab_al-Nasr_(Cairo),When established,None
3,Bab_al-Nasr_(Cairo),Who built it,None
4,Bab_al-Nasr_(Cairo),Which ruler commissioned,None
...,...,...,...
2571,Abu_Ghurab,Which ruler commissioned,None
2572,Abu_Ghurab,Which architectural style,None
2573,Abu_Ghurab,What original name,None
2574,Abu_Ghurab,What primary material,None


In [49]:
const_qa_data= data.iloc[:,:2].drop_duplicates().merge(const_qa_data, on='Concept', how='left')
const_qa_data

,Index,Concept,Question,Answer
0,73.0,Bab_al-Nasr_(Cairo),What Cultural Category does the object in the ...,Famous Landmarks
1,73.0,Bab_al-Nasr_(Cairo),Where exactly located,None
2,73.0,Bab_al-Nasr_(Cairo),When established,None
3,73.0,Bab_al-Nasr_(Cairo),Who built it,None
4,73.0,Bab_al-Nasr_(Cairo),Which ruler commissioned,None
...,...,...,...,...
2589,351.0,Abu_Ghurab,Which ruler commissioned,None
2590,351.0,Abu_Ghurab,Which architectural style,None
2591,351.0,Abu_Ghurab,What original name,None
2592,351.0,Abu_Ghurab,What primary material,None


In [50]:
const_qa_data.to_csv("const_qa_data.csv")